In [10]:
!pip install tensorflow-gpu
!pip install gym[all]
!pip install keras
!pip install keras-rl2

Reason for being yanked: re-release with new wheels
  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\bogda\anaconda3\envs\env1\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\bogda\\AppData\\Local\\Temp\\pip-install-4xdi5aub\\box2d-py_7121fba241b1421d83a6590fb34d1cf6\\setup.py'"'"'; __file__='"'"'C:\\Users\\bogda\\AppData\\Local\\Temp\\pip-install-4xdi5aub\\box2d-py_7121fba241b1421d83a6590fb34d1cf6\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\bogda\AppData\Local\Temp\pip-wheel-gg6veix0'
       cwd: C:\Users\bogda\AppData\Local\Temp\pip-install-4xdi5aub\box2d-py_7121fba241b1421d83a6590fb34d1cf6\
  Complete output (16 lines):
  Using setuptools (version 49.6.0.post20210108).
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-

  Created wheel for atari-py: filename=atari_py-0.2.6-cp39-cp39-win_amd64.whl size=1155035 sha256=0b10e6f5d9fb53e54e9311b245bbfcbfb55ee4d8da928ca7ba27eef1f05bce0e
  Stored in directory: c:\users\bogda\appdata\local\pip\cache\wheels\5b\1f\fb\f8135d758ce2cd876eefa28f4320702169566d28de4d2e8aa6
  Running setup.py clean for box2d-py
  Running setup.py clean for mujoco-py
Successfully built atari-py
Failed to build box2d-py mujoco-py
    Running setup.py install for mujoco-py: started
    Running setup.py install for mujoco-py: finished with status 'error'


In [5]:
import gym
import random

In [6]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [7]:
episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode: {} Score: {}'.format(episode, score))

Episode: 1 Score: 21.0
Episode: 2 Score: 15.0
Episode: 3 Score: 20.0
Episode: 4 Score: 22.0
Episode: 5 Score: 38.0
Episode: 6 Score: 41.0
Episode: 7 Score: 12.0
Episode: 8 Score: 20.0
Episode: 9 Score: 14.0
Episode: 10 Score: 11.0


In [19]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [20]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
model = build_model(states, actions)

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [23]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [24]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [25]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)

    1/10000 [..............................] - ETA: 6:12 - reward: 1.0000

C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
111 episodes - episode_reward: 90.036 [8.000, 200.000] - loss: 3.488 - mae: 19.453 - mean_q: 39.336

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: 1.0000
56 episodes - episode_reward: 177.161 [139.000, 200.000] - loss: 7.457 - mae: 38.977 - mean_q: 78.526

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 54s 5ms/step - reward: 1.0000
54 episodes - episode_reward: 184.519 [143.000, 200.000] - loss: 6.016 - mae: 40.177 - mean_q: 80.857

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
52 episodes - episode_reward: 191.327 [138.000, 200.000] - loss: 4.436 - mae: 38.536 - mean_q: 77.409

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 1.0000
done, took 264.951 seconds


In [26]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 180.000, steps: 180
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 172.000, steps: 172
Episode 7: reward: 172.000, steps: 172
Episode 8: reward: 152.000, steps: 152
Episode 9: reward: 183.000, steps: 183
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 170.000, steps: 170
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 160.000, steps: 160
Episode 17: reward: 180.000, steps: 180
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 157.000, steps: 157
Episode 20: reward: 193.000, steps: 193
Episode 21: reward: 166.000, steps: 166
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [28]:
_ = dqn.test(env, nb_episodes=100, visualize=True)

Testing for 100 episodes ...
Episode 1: reward: 179.000, steps: 179
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 192.000, steps: 192
Episode 4: reward: 186.000, steps: 186
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 154.000, steps: 154
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 166.000, steps: 166
Episode 13: reward: 196.000, steps: 196
Episode 14: reward: 163.000, steps: 163
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 168.000, steps: 168
Episode 17: reward: 191.000, steps: 191
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 150.000, steps: 150
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 182.000, steps: 182
Episode 23: reward: 163.000, steps: 163
Episode 24: reward: 145.000, steps: 145
Episode 25: reward: 

In [29]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [30]:
del model
del dqn
del env

In [31]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model=build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [32]:
dqn.load_weights('dqn_weights.h5f')

In [33]:
dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


Exception ignored in: <function Viewer.__del__ at 0x000002220586F160>
Traceback (most recent call last):
  File "C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\bogda\anaconda3\envs\env1\lib\site-packages\pyglet\window\__init__.py", line 848, in close
    app.windows.remove(self)
  File "C:\Users\bogda\anaconda3\envs\env1\lib\_weakrefset.py", line 110, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000022205760DB0; to 'Win32Window' at 0x00000221EF76E790>


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 150.000, steps: 150


In [35]:
env.close()

AttributeError: 'DQNAgent' object has no attribute 'quit'